# CMIP5: May Distribution Fits, 2041-2070

Downscaled CMIP5 simulation results were obtained using two downscaling methods period 2041-2070. These values have been collated into monthly collections of daily precipitation by grid cell and by group. Four groups were created using PCA and K-means clustering. The groups identified from PRISM data were also attributed to the grid locations.

CMIP5 data will be processed by grid cell and regions will not be used for fitting distributions. There are 4, 30-yr climate periods and expect different regionalization for each climate period. Consequently, the most direct approach will be to use the grid cells as independent regions and then we get the automatic variation.

- LOCA: 1/16 degree grid cells
- BCCA: 1/8 degree grid cells == NLDAS2 grid

## Parameters and Library Loading

In [ ]:
oldw <- getOption("warn")

In [ ]:
options(warn = -1)

In [ ]:
library(feather)
library(moments)
library(dplyr)
library(fitdistrplus)
library(gendist)
library(mixtools)
library(xlsx)
library(r2excel)
library("IRdisplay")

LOCA grid to show which cells, **Grid_Id**, values are actually needed.

In [ ]:
display_png(file="//augustine.space.swri.edu/jdrive/Groundwater/R8937_Stochastic_CC_Recharge/Data/JNotes/Images/PNG/Jan_LOCA_grid_PRISM_regions_.png")

In [ ]:
LOCA_IDs <- c( 136, 137, 120, 121, 122, 123, 104, 105, 106, 107, 108, 109,
               90, 91, 92, 93, 94, 76, 77, 78, 79, 62, 63, 64 )

BCCA grid to show which BCCA cells are needed

In [ ]:
display_png(file="//augustine.space.swri.edu/jdrive/Groundwater/R8937_Stochastic_CC_Recharge/Data/JNotes/Images/PNG/Jan_BCCA_grid_PRISM_regions_.png")

In [ ]:
BCCA_IDs <- c( 200, 201, 202, 192, 193, 194, 195, 185, 186, 187, 179 )

In [ ]:
NBCGrid <- length( BCCA_IDs )
NLOGrid <- length( LOCA_IDs )
cat( "LOCA grid cells of interest: ", NLOGrid, " and BCCA grid cells of interest: ", NBCGrid )

In [ ]:
setwd("//augustine.space.swri.edu/jdrive/Groundwater/R8937_Stochastic_CC_Recharge/Data/R/Working/CMIP5_2041_PDepth")

Specify some parameters, primarily filenames

Have two different downscaled grids and so have a separate file for each.

In [ ]:
feLONAM <- "May_WetDays_LOCA_Grp_2041-2070.feather"

In [ ]:
fBCNAM <- "May_WetDays_BCCA_Grp_2041-2070.feather"

In [ ]:
dfLOMay <- read_feather(feLONAM)

In [ ]:
dfBCMay <- read_feather(fBCNAM)

## May Distribution Fits

Fit distributions for each defined grid cell in the header.

There are 4 steps in fitting distributions (Ricci, 2005):

1. Model/function choice: hypothesize families of distributions;
2. Estimate parameters;
3. Evaluate quality of fit;
4. Goodness of fit statistical tests.

We will use two, pre-selected distributions for fitting. The first distribution is the 2-parameter gamma distribution which is often used for precipitation depth. This distribution has some advantages in terms of fitting ability because of the two parameters relative to the exponential distribution which is one parameter.

The second distribution to try is the mixed exponential distribution which is a probability mixture of two one-parameter exponential distributions. It provides for the superposition of two ordinary exponential distributions whose means are $\mu_{1}$ and $\mu_{2}$. It provides a better representation of the frequencies of teh very largest precipitation amounts

\begin{equation*}
    f(x) = \frac{ \alpha }{\mu_{1}} \exp \left[ \frac{-x}{\mu_{1}} \right] + \frac{ 1 - \alpha }
    {\mu_{2}} \exp \left[ \frac{-x}{\mu_{2}} \right]
\end{equation*}

\begin{equation*}
    \mu = \alpha \mu_{1} + \left( 1 - \alpha \right) \mu_{2}
\end{equation*}

\begin{equation*}
    \sigma^{2} = \alpha \mu_{1}^{2} + \left( 1 - \alpha \right) \mu_{2}^{2} + \alpha 
    \left( 1 - \alpha \right) \left( \mu_{1} - \mu_{2} \right)^{2}
\end{equation*}

Use a DataFrame to track our results and then output to a spreadsheet.

### Grid Cell Distributions

In [ ]:
MonLODistDF <- data.frame( gridno=rep(NA,NLOGrid), region=rep(NA,NLOGrid), GM_shape=rep(NA,NLOGrid), 
                           GM_rate=rep(NA,NLOGrid), GM_llike=rep(NA,NLOGrid), GM_mean=rep(NA,NLOGrid),
                           GM_var=rep(NA,NLOGrid), GM_KSstat=rep(NA,NLOGrid), GM_KSp=rep(NA,NLOGrid),
                           ME_rate1=rep(NA,NLOGrid), ME_rate2=rep(NA,NLOGrid), ME_lambda=rep(NA,NLOGrid),
                           ME_mean1=rep(NA,NLOGrid), ME_mean2=rep(NA,NLOGrid), ME_mean=rep(NA,NLOGrid),
                           ME_var1=rep(NA,NLOGrid), ME_var2=rep(NA,NLOGrid), ME_var=rep(NA,NLOGrid),
                           ME_llike=rep(NA,NLOGrid), ME_KSstat=rep(NA,NLOGrid), ME_KSp=rep(NA,NLOGrid),
                           stringsAsFactors=FALSE )

In [ ]:
MonBCDistDF <- data.frame( gridno=rep(NA,NBCGrid), region=rep(NA,NBCGrid), GM_shape=rep(NA,NBCGrid), 
                           GM_rate=rep(NA,NBCGrid), GM_llike=rep(NA,NBCGrid), GM_mean=rep(NA,NBCGrid),
                           GM_var=rep(NA,NBCGrid), GM_KSstat=rep(NA,NBCGrid), GM_KSp=rep(NA,NBCGrid),
                           ME_rate1=rep(NA,NBCGrid), ME_rate2=rep(NA,NBCGrid), ME_lambda=rep(NA,NBCGrid),
                           ME_mean1=rep(NA,NBCGrid), ME_mean2=rep(NA,NBCGrid), ME_mean=rep(NA,NBCGrid),
                           ME_var1=rep(NA,NBCGrid), ME_var2=rep(NA,NBCGrid), ME_var=rep(NA,NBCGrid),
                           ME_llike=rep(NA,NBCGrid), ME_KSstat=rep(NA,NBCGrid), ME_KSp=rep(NA,NBCGrid),
                           stringsAsFactors=FALSE )

In [ ]:
setwd("//augustine.space.swri.edu/jdrive/Groundwater/R8937_Stochastic_CC_Recharge/Data/R/Working/CMIP5_2041_PDepth/Plots")

In [ ]:
LOGridVct <- as.vector( 1:NLOGrid )
for (iI in LOGridVct) {
    # setup
    cGridId <- LOCA_IDs[iI]
    dfCGrid <- dfLOMay %>% filter( Grid_Id == cGridId )
    cReg <- dfCGrid$PRegion_Id[1]
    # gamma
    fd_GM_MayA <- fitdist( dfCGrid$Precip_mm, "gamma" )
    fdGM_MayA_shape <- fd_GM_MayA$estimate[["shape"]]
    fdGM_MayA_rate <- fd_GM_MayA$estimate[["rate"]]
    fdGM_MayA_llike <- fd_GM_MayA$loglik
    fdGM_MayA_mean <- fdGM_MayA_shape / fdGM_MayA_rate
    fdGM_MayA_var <- fdGM_MayA_shape / ( fdGM_MayA_rate ^ 2 )
    ksRes <- ks.test( dfCGrid$Precip_mm, "pgamma", shape=fdGM_MayA_shape, rate=fdGM_MayA_rate )
    fdGM_MayA_KSStat <- ksRes$statistic
    fdGM_MayA_KSp <- ksRes$p.value
    # mixed exponential
    fd_ME_MayA <- expRMM_EM( dfCGrid$Precip_mm, d=NULL, lambda=c(0.05, 1-0.05),
                             rate=c(1.0/median(dfCGrid$Precip_mm), 1.0/mean(dfCGrid$Precip_mm)), k=2,
                             complete="xz", epsilon=1e-08, maxit=1000, verb=FALSE )
    fdME_MayA_rate1 <- fd_ME_MayA$rate[[1]]
    fdME_MayA_rate2 <- fd_ME_MayA$rate[[2]]
    fdME_MayA_lambda <- fd_ME_MayA$lambda[[1]]
    fdME_MayA_mean1 <- 1.0 / fdME_MayA_rate1
    fdME_MayA_mean2 <- 1.0 / fdME_MayA_rate2
    fdME_MayA_mean <- ( (fdME_MayA_lambda * fdME_MayA_mean1) + 
                        ( ( 1.0 - fdME_MayA_lambda) * fdME_MayA_mean2 ) )
    fdME_MayA_var1 <- 1.0 / (fdME_MayA_rate1 ^ 2)
    fdME_MayA_var2 <- 1.0 / (fdME_MayA_rate2 ^ 2)
    fdME_MayA_var <- ( (fdME_MayA_lambda * ( fdME_MayA_mean1 ^ 2 ) ) + 
                       ( ( 1.0 - fdME_MayA_lambda) * (fdME_MayA_mean2 ^ 2) ) + 
                       ( fdME_MayA_lambda * ( 1.0 - fdME_MayA_lambda) * 
                           ( fdME_MayA_mean1 - fdME_MayA_mean2 )^2 ) )
    fdME_MayA_llike <- fd_ME_MayA$loglik
    tvals <- rexpmix( length(dfCGrid$Precip_mm), fd_ME_MayA$lambda, fd_ME_MayA$rate )
    ksResME <- ks.test( dfCGrid$Precip_mm, tvals )
    fdME_MayA_KSStat <- ksResME$statistic
    fdME_MayA_KSp <- ksResME$p.value
    # save the values
    MonLODistDF[iI, ] <- list( cGridId, cReg, fdGM_MayA_shape, fdGM_MayA_rate, fdGM_MayA_llike, fdGM_MayA_mean,
                            fdGM_MayA_var, fdGM_MayA_KSStat, fdGM_MayA_KSp, fdME_MayA_rate1,
                            fdME_MayA_rate2, fdME_MayA_lambda, fdME_MayA_mean1, fdME_MayA_mean2,
                            fdME_MayA_mean, fdME_MayA_var1, fdME_MayA_var2, fdME_MayA_var,
                            fdME_MayA_llike, fdME_MayA_KSStat, fdME_MayA_KSp )
    # plots section
    wMayAMax <- max( max( dfCGrid$Precip_mm ), max( tvals ) )
    PName <- paste("May_LOCA_G", cGridId, "_QQ.png")
    png(filename=PName)
    qqplot( tvals, dfCGrid$Precip_mm, col="green", xlab="Theoretical Quantiles",
            ylab="Sample Quantiles", main="May All Q-Q Plot",
            xlim=c(0,wMayAMax), ylim=c(0,wMayAMax) )
    abline( 0, 1)
    dev.off()
    x <- seq(0,wMayAMax,1)
    plot.legend <- c("Fitted Distribution", "Data Sample" )
    PName <- paste("May_LOCA_G", cGridId, "_ECDFs.png")
    png(filename=PName)
    plot(ecdf(tvals),
         xlab="Precip Depth (mm)", ylab="Cumulative Density",
         col="blue", main="Comparison of Empirical and Fitted CDFs" )
    plot(ecdf(dfCGrid$Precip_mm), col="green", add=TRUE )
    legend('bottomright', plot.legend, lty=1, col=c("blue", "green"))
    dev.off()
    maxP <- max(dfCGrid$Precip_mm)
    xVals <- seq(1, maxP, 1)
    pdVals <- dmixt( xVals, phi=fdME_MayA_lambda, spec1="exp", arg1=list(rate=fdME_MayA_rate1),
                     spec2="exp", arg2=list(rate=fdME_MayA_rate2) )
    PName <- paste("May_LOCA_G", cGridId, "_HistFitME.png")
    png(filename=PName)
    hist( dfCGrid$Precip_mm, freq=FALSE, col="lightsteelblue", 
          xlab="Precipitation (mm/day)", ylab="Probability Density",
          main="May All Histogram vs Fitted Mixed Exponential Distribution" )
    lines( xVals, pdVals, lwd=2, lty=1, col="firebrick", add=TRUE)
    dev.off()
}

In [ ]:
BCGridVct <- as.vector( 1:NBCGrid )
for (iI in BCGridVct) {
    # setup
    cGridId <- BCCA_IDs[iI]
    dfCGrid <- dfBCMay %>% filter( Grid_Id == cGridId )
    cReg <- dfCGrid$PRegion_Id[1]
    # gamma
    fd_GM_MayA <- fitdist( dfCGrid$Precip_mm, "gamma" )
    fdGM_MayA_shape <- fd_GM_MayA$estimate[["shape"]]
    fdGM_MayA_rate <- fd_GM_MayA$estimate[["rate"]]
    fdGM_MayA_llike <- fd_GM_MayA$loglik
    fdGM_MayA_mean <- fdGM_MayA_shape / fdGM_MayA_rate
    fdGM_MayA_var <- fdGM_MayA_shape / ( fdGM_MayA_rate ^ 2 )
    ksRes <- ks.test( dfCGrid$Precip_mm, "pgamma", shape=fdGM_MayA_shape, rate=fdGM_MayA_rate )
    fdGM_MayA_KSStat <- ksRes$statistic
    fdGM_MayA_KSp <- ksRes$p.value
    # mixed exponential
    fd_MEB_MayA <- expRMM_EM( dfCGrid$Precip_mm, d=NULL, lambda=c(0.05, 1-0.05),
                             rate=c(1.0/median(dfCGrid$Precip_mm), 1.0/mean(dfCGrid$Precip_mm)), k=2,
                             complete="xz", epsilon=1e-08, maxit=1000, verb=FALSE )
    fdMEB_MayA_rate1 <- fd_MEB_MayA$rate[[1]]
    fdMEB_MayA_rate2 <- fd_MEB_MayA$rate[[2]]
    fdMEB_MayA_lambda <- fd_MEB_MayA$lambda[[1]]
    fdMEB_MayA_mean1 <- 1.0 / fdMEB_MayA_rate1
    fdMEB_MayA_mean2 <- 1.0 / fdMEB_MayA_rate2
    fdMEB_MayA_mean <- ( (fdMEB_MayA_lambda * fdMEB_MayA_mean1) + 
                        ( ( 1.0 - fdMEB_MayA_lambda) * fdMEB_MayA_mean2 ) )
    fdMEB_MayA_var1 <- 1.0 / (fdMEB_MayA_rate1 ^ 2)
    fdMEB_MayA_var2 <- 1.0 / (fdMEB_MayA_rate2 ^ 2)
    fdMEB_MayA_var <- ( (fdMEB_MayA_lambda * ( fdMEB_MayA_mean1 ^ 2 ) ) + 
                       ( ( 1.0 - fdMEB_MayA_lambda) * (fdMEB_MayA_mean2 ^ 2) ) + 
                       ( fdMEB_MayA_lambda * ( 1.0 - fdMEB_MayA_lambda) * 
                           ( fdMEB_MayA_mean1 - fdMEB_MayA_mean2 )^2 ) )
    fdMEB_MayA_llike <- fd_MEB_MayA$loglik
    tvals <- rexpmix( length(dfCGrid$Precip_mm), fd_MEB_MayA$lambda, fd_MEB_MayA$rate )
    ksResME <- ks.test( dfCGrid$Precip_mm, tvals )
    fdMEB_MayA_KSStat <- ksResME$statistic
    fdMEB_MayA_KSp <- ksResME$p.value
    # save the values
    MonBCDistDF[iI, ] <- list( cGridId, cReg, fdGM_MayA_shape, fdGM_MayA_rate, fdGM_MayA_llike, fdGM_MayA_mean,
                            fdGM_MayA_var, fdGM_MayA_KSStat, fdGM_MayA_KSp, fdMEB_MayA_rate1,
                            fdMEB_MayA_rate2, fdMEB_MayA_lambda, fdMEB_MayA_mean1, fdMEB_MayA_mean2,
                            fdMEB_MayA_mean, fdMEB_MayA_var1, fdMEB_MayA_var2, fdMEB_MayA_var,
                            fdMEB_MayA_llike, fdMEB_MayA_KSStat, fdMEB_MayA_KSp )
    # plots section
    wMayAMax <- max( max( dfCGrid$Precip_mm ), max( tvals ) )
    PName <- paste("May_BCCA_G", cGridId, "_QQ.png")
    png(filename=PName)
    qqplot( tvals, dfCGrid$Precip_mm, col="green", xlab="Theoretical Quantiles",
            ylab="Sample Quantiles", main="May All Q-Q PBCt",
            xlim=c(0,wMayAMax), ylim=c(0,wMayAMax) )
    abline( 0, 1)
    dev.off()
    x <- seq(0,wMayAMax,1)
    plot.legend <- c("Fitted Distribution", "Data Sample" )
    PName <- paste("May_BCCA_G", cGridId, "_ECDFs.png")
    png(filename=PName)
    plot(ecdf(tvals),
         xlab="Precip Depth (mm)", ylab="Cumulative Density",
         col="blue", main="Comparison of Empirical and Fitted CDFs" )
    plot(ecdf(dfCGrid$Precip_mm), col="green", add=TRUE )
    legend('bottomright', plot.legend, lty=1, col=c("blue", "green"))
    dev.off()
    maxP <- max(dfCGrid$Precip_mm)
    xVals <- seq(1, maxP, 1)
    pdVals <- dmixt( xVals, phi=fdMEB_MayA_lambda, spec1="exp", arg1=list(rate=fdMEB_MayA_rate1),
                     spec2="exp", arg2=list(rate=fdMEB_MayA_rate2) )
    PName <- paste("May_BCCA_G", cGridId, "_HistFitME.png")
    png(filename=PName)
    hist( dfCGrid$Precip_mm, freq=FALSE, col="lightsteelblue", 
          xlab="Precipitation (mm/day)", ylab="Probability Density",
          main="May All Histogram vs Fitted Mixed Exponential Distribution" )
    lines( xVals, pdVals, lwd=2, lty=1, col="firebrick", add=TRUE)
    dev.off()
}

## Output Stats and Distribution Fits to Spreadsheet

In [ ]:
setwd("//augustine.space.swri.edu/jdrive/Groundwater/R8937_Stochastic_CC_Recharge/Data/R/Working/CMIP5_2041_PDepth")

In [ ]:
outputDF_xlsx <- createWorkbook()

In [ ]:
MayLO_xlsx <- createSheet(wb=outputDF_xlsx, sheetName="May_LOCA")
MayBC_xlsx <- createSheet(wb=outputDF_xlsx, sheetName="May_BCCA")

In [ ]:
addDataFrame( x=MonLODistDF, sheet=MayLO_xlsx )
addDataFrame( x=MonBCDistDF, sheet=MayBC_xlsx )

In [ ]:
saveWorkbook( outputDF_xlsx, file="CMIP5_2070_MayDistFits.xlsx" )

In [ ]:
options(warn = oldw)